# Clone Source Code

In [1]:
!git clone https://github.com/hein-nkhh/unilm.git
%cd unilm/beit3

Cloning into 'unilm'...
remote: Enumerating objects: 11122, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 11122 (delta 31), reused 18 (delta 18), pack-reused 11079 (from 4)
Receiving objects: 100% (11122/11122), 75.39 MiB | 35.87 MiB/s, done.
Resolving deltas: 100% (5248/5248), done.
/kaggle/working/unilm/beit3


# Import thư viện

In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.6/444.6 kB 10.5 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 15.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.2/398.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.3/216.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.3 MB/s eta

In [3]:
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image
import time
import pickle
import torch
from IPython.display import clear_output
import os
import torch.nn.functional as F
from transformers import AutoTokenizer
from modeling_finetune import beit3_large_patch16_384_retrieval
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.cuda.amp import autocast
from transformers import XLMRobertaTokenizer
import json
import cv2
from huggingface_hub import HfApi
import math
import torch.multiprocessing as mp
from torch.utils.data import Dataset, DataLoader
import os
import time
import json
import cv2
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    XLMRobertaTokenizer
)
from modeling_finetune import beit3_large_patch16_384_retrieval
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm import tqdm, notebook
import shutil


In [4]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True,                 # ép ghi đè cấu hình cũ (rất quan trọng trong notebook)
)

logger = logging.getLogger("Embedd Frame")
logger.info("Xin chào")

2025-09-05 10:30:38,885 - Embedd Frame - INFO - Xin chào


# Cài đặt Device Torch

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Extract Embedding

In [50]:
%%writefile multi_gpu_extract.py
import os
import av
import time
import json
import cv2
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    XLMRobertaTokenizer
)
from modeling_finetune import beit3_large_patch16_384_retrieval
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm import tqdm, notebook
import shutil
from embedding_io import EmbeddingIO
from hf_uploader import HFUploader

# class VideoFrameDataset(Dataset):
#     def __init__(self, video_path, save_dir="frames", transform=None, show_progress=True):
#         """
#         Dataset cho một video duy nhất, preload toàn bộ frame ra ảnh và trả về path.

#         Args:
#             video_path (str): đường dẫn đến file video
#             save_dir (str): thư mục lưu frame
#             transform: torchvision transforms áp dụng khi load frame
#             show_progress (bool): có hiển thị tqdm progress bar hay không
#         """
#         self.video_path = video_path
#         self.save_dir = save_dir
#         self.transform = transform
#         self.frame_paths = []

#         os.makedirs(self.save_dir, exist_ok=True)

#         cap = cv2.VideoCapture(self.video_path)
#         if not cap.isOpened():
#             raise ValueError(f"⚠️ Error loading video: {self.video_path}")

#         total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # tổng số frame
#         pbar = tqdm(total=total_frames, desc=f"Extracting {video_path}", disable=not show_progress)

#         idx = 0
#         while True:
#             ret, frame = cap.read()
#             if not ret:
#                 break
#             frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#             # path lưu frame
#             frame_path = os.path.join(self.save_dir, f"frame_{idx:06d}.jpg")
#             cv2.imwrite(frame_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # lưu lại dưới dạng jpg
#             self.frame_paths.append(frame_path)

#             idx += 1
#             if idx == 2:
#                 break
#             pbar.update(1)

#         cap.release()
#         pbar.close()


#         if len(self.frame_paths) == 0:
#             raise ValueError(f"⚠️ No frames extracted from video: {self.video_path}")

#     def __len__(self):
#         return len(self.frame_paths)

#     def __getitem__(self, idx):
#         frame_path = self.frame_paths[idx]

#         if self.transform:
#             import PIL.Image
#             img = PIL.Image.open(frame_path).convert("RGB")
#             img = self.transform(img)
#             return img, frame_path

#         return frame_path

import os
from torch.utils.data import Dataset
from tqdm import tqdm
import av
from PIL import Image

class VideoFrameDataset(Dataset):
    def __init__(self, video_path, save_dir="frames", transform=None, show_progress=True):
        """
        Dataset cho một video duy nhất, preload toàn bộ frame ra ảnh và trả về path.

        Args:
            video_path (str): đường dẫn đến file video
            save_dir (str): thư mục lưu frame
            transform: torchvision transforms áp dụng khi load frame
            show_progress (bool): có hiển thị tqdm progress bar hay không
        """
        self.video_path = video_path
        self.save_dir = save_dir
        self.transform = transform
        self.frame_paths = []

        os.makedirs(self.save_dir, exist_ok=True)

        try:
            container = av.open(self.video_path)
        except av.AVError as e:
            raise ValueError(f"⚠️ Error loading video: {self.video_path}\n{e}")

        total_frames = container.streams.video[0].frames
        pbar = tqdm(total=total_frames, desc=f"Extracting {video_path}", disable=not show_progress)

        idx = 0
        for frame in container.decode(video=0):
            img = frame.to_image()  # PIL Image
            frame_path = os.path.join(self.save_dir, f"frame_{idx:06d}.jpg")
            img.save(frame_path)
            self.frame_paths.append(frame_path)
            idx += 1
            if idx == 10:
                break
            pbar.update(1)

        pbar.close()

        if len(self.frame_paths) == 0:
            raise ValueError(f"⚠️ No frames extracted from video: {self.video_path}")

    def __len__(self):
        return len(self.frame_paths)

    def __getitem__(self, idx):
        frame_path = self.frame_paths[idx]

        if self.transform:
            img = Image.open(frame_path).convert("RGB")
            img = self.transform(img)
            return img, frame_path

        return frame_path

def collate_fn(batch):
    """Bỏ qua sample lỗi (None)"""
    batch = [x for x in batch if x[0] is not None]
    if not batch:
        return None, None
    images, paths = zip(*batch)
    return torch.stack(images, dim=0), list(paths)

def extract_embeddings_dataloader(dataset, model, transform, device, batch_size=64, num_workers=4):
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        collate_fn=collate_fn,
        prefetch_factor=3,
        pin_memory=True
    )

    embeddings = []
    ids = []

    model.eval()
    start_extract_embedding = time.time()

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="🔄 Extracting embeddings (DataLoader)"):
            batch_tensor, batch_ids = batch
            if batch_tensor is None:
                continue

            batch_tensor = batch_tensor.to(device, non_blocking=True)

            with autocast():
                vision_cls_batch = model(image=batch_tensor, only_infer=True)[0]  # (B, D)
                vision_norm_batch = F.normalize(vision_cls_batch, p=2, dim=-1)

            embeddings.extend([emb.cpu() for emb in vision_norm_batch])
            ids.extend(batch_ids)

            del batch_tensor, vision_cls_batch, vision_norm_batch
            torch.cuda.empty_cache()

    image_embeddings = torch.stack(embeddings, dim=0)
    end_extract_embedding = time.time()
    print(f"Thời gian extract embedding: {end_extract_embedding-start_extract_embedding}")

    return image_embeddings, ids

def upload_embeddings(local_file: str, repo_id: str, token: str,
                       repo_type: str = "dataset", path_in_repo: str = None):
    """
    Upload một file từ local lên Hugging Face repo và trả về URL trong repo.
    Args:
        local_file: đường dẫn file ở local
        repo_id: repo_id trên Hugging Face (vd: "username/my-dataset")
        token: Hugging Face token
        repo_type: loại repo ("model", "dataset", "space")
        path_in_repo: đường dẫn lưu trong repo (mặc định = tên file)
    """
    uploader = HFUploader(token=token)
    uploader.upload_file(
        local_path=local_file,
        repo_id=repo_id,
        repo_type=repo_type,
        path_in_repo=path_in_repo
    )

    if path_in_repo is None:
        import os
        path_in_repo = os.path.basename(local_file)

    # URL file trong repo
    base_url = f"https://huggingface.co/{repo_id}/resolve/main"
    return f"{base_url}/{path_in_repo}"

def process_video(gpu_id, video_queue, token):
    device = torch.device(f"cuda:{gpu_id}")
    print(f"🚀 Worker GPU {gpu_id} started")

    # Load model
    tokenizer = XLMRobertaTokenizer("/kaggle/input/beit3_base_retrieval/pytorch/default/2/beit3.spm")
    ckpt = "/kaggle/input/beit3_base_retrieval/pytorch/default/2/beit3_large_patch16_384_coco_retrieval.pth"
    model = beit3_large_patch16_384_retrieval(pretrained=False)
    state_dict = torch.load(ckpt, map_location=device)
    model.load_state_dict(state_dict["model"], strict=False)
    model = model.to(device).eval()

    # Transform
    transform  = transforms.Compose([
        transforms.Resize((384, 384), interpolation=3), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
    ])

    while True:
        try:
            video_path = video_queue.get(timeout=5)   # chờ tối đa 5s
        except Exception:
            print(f"✅ GPU {gpu_id} done (no more videos).")
            break

        vid_name = os.path.splitext(os.path.basename(video_path))[0]
        save_dir = f"/kaggle/working/frames_{vid_name}"
        output_path = f"/kaggle/working/embeddings/{vid_name}"
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        print(f"🎬 GPU {gpu_id} processing {vid_name}")

        try:
            dataset = VideoFrameDataset(video_path=video_path, save_dir=save_dir, transform=transform)
            # dataset.frame_paths = dataset.frame_paths[:1000]
            
            embeddings, ids = extract_embeddings_dataloader(
                dataset, model, transform, device, batch_size=64, num_workers=4
            )
            
            # 👉 convert sang float16 để tiết kiệm dung lượng
            embeddings = embeddings.half().cpu()

            io = EmbeddingIO(default_fmt="pkl")
            file_save = io.save(embeddings, output_path)
            huggingface_dir = os.path.basename(os.path.dirname(os.path.dirname(video_path)))
            
            path_in_repo = os.path.join(huggingface_dir, os.path.basename(file_save))
            print(path_in_repo)
            repo_id = 'AIC3HUIT/AIC_2025'
            for i in range(3):
                try:
                    upload_embeddings(local_file=file_save, 
                                           path_in_repo=path_in_repo,
                                           repo_id=repo_id, 
                                            token=token)
                    print(f"Đẩy file {file_save} thành công lên huggingface")
                except:
                    print(f"Đẩy file {file_save} bị lỗi... Đang thử lại lần {i}")
                    time.sleep(1 + i)
        except Exception as e:
            print(f"Lỗi khi xử lý {video_path} {e}")
        finally:
            # luôn xoá frames folder sau khi xử lý
            shutil.rmtree(save_dir, ignore_errors=True)
            print(f"🗑️ Deleted frames folder: {save_dir}")


Overwriting multi_gpu_extract.py


# Lưu Embeddings

In [7]:
%%writefile embedding_io.py
import torch
import pickle

class EmbeddingIO:
    def __init__(self, default_fmt: str = "pkl"):
        """
        Class để lưu / load embeddings.
        Args:
            default_fmt (str): định dạng mặc định ("pkl" hoặc "pt")
        """
        self.default_fmt = default_fmt.lower()

    def save(self, embeddings, file_path: str, fmt: str = None):
        """
        Lưu embeddings ra file pkl hoặc pt.
        Args:
            embeddings: torch.Tensor hoặc numpy.ndarray
            file_path (str): đường dẫn file (không cần đuôi)
            fmt (str): "pkl" hoặc "pt". Nếu None -> dùng default_fmt
        """
        fmt = (fmt or self.default_fmt).lower()
        if fmt == "pkl":
            file_path = file_path + ".pkl"
            with open(file_path, "wb") as f:
                pickle.dump(embeddings, f)
            print(f"✅ Saved {file_path}")
        elif fmt == "pt":
            file_path = file_path + ".pt"
            torch.save(embeddings, file_path)
            print(f"✅ Saved {file_path}")
        else:
            raise ValueError("fmt phải là 'pkl' hoặc 'pt'")
        return file_path

    def load(self, file_path: str, fmt: str = None):
        """
        Load embeddings từ file pkl hoặc pt.
        Args:
            file_path (str): đường dẫn file (không cần đuôi)
            fmt (str): "pkl" hoặc "pt". Nếu None -> dùng default_fmt
        Returns:
            torch.Tensor hoặc numpy.ndarray
        """
        fmt = (fmt or self.default_fmt).lower()
        if fmt == "pkl":
            with open(file_path + ".pkl", "rb") as f:
                return pickle.load(f)
        elif fmt == "pt":
            return torch.load(file_path + ".pt")
        else:
            raise ValueError("fmt phải là 'pkl' hoặc 'pt'")

Writing embedding_io.py


# Push Embedding

In [8]:
%%writefile hf_uploader.py

from huggingface_hub import HfApi


class HFUploader:
    def __init__(self, token: str = None):
        """
        Khởi tạo uploader.
        Nếu có token -> dùng để xác minh.
        """
        self.api = HfApi()
        self.token = token

    def upload_file(self, local_path: str, repo_id: str, path_in_repo: str = None, repo_type: str = "model"):
        """
        Upload một file lên Hugging Face.
        """
        if path_in_repo is None:
            import os
            path_in_repo = os.path.basename(local_path)

        return self.api.upload_file(
            path_or_fileobj=local_path,
            path_in_repo=path_in_repo,
            repo_id=repo_id,
            repo_type=repo_type,
            token=self.token
        )

    def upload_folder(self, local_folder: str, repo_id: str, repo_type: str = "dataset", path_in_repo: str = ""):
        """
        Upload một thư mục lên Hugging Face, có thể chỉ định thư mục con trong repo.
        """
        return self.api.upload_folder(
            folder_path=local_folder,
            path_in_repo=path_in_repo,   # thư mục con trong repo, vd: "data/"
            repo_id=repo_id,
            repo_type=repo_type,
            token=self.token
        )

Writing hf_uploader.py


# Prepare video path file

In [9]:
import os
import math

class VideoPathPrepare:
    VIDEO_EXTENSIONS = {".mp4", ".avi", ".mov", ".mkv", ".flv", ".wmv", ".webm"}

    def __init__(self, root_dirs, mode="subdir"):
        """
        Args:
            root_dirs (str hoặc list[str]): 1 hoặc nhiều thư mục
            mode (str): 
                - "subdir": root_dirs chứa nhiều thư mục con "Videos*"
                - "root": root_dirs chính là thư mục chứa video
        """
        if isinstance(root_dirs, str):
            root_dirs = [root_dirs]
        self.root_dirs = root_dirs

        if mode not in {"subdir", "root"}:
            raise ValueError("mode phải là 'subdir' hoặc 'root'")
        self.mode = mode

        self.videos = self._list_videos()

    def _list_videos(self):
        video_files = []
        for root_dir in self.root_dirs:
            if self.mode == "subdir":
                # 🔹 Tìm thư mục con 'Videos*'
                subdirs = [
                    d for d in os.listdir(root_dir)
                    if os.path.isdir(os.path.join(root_dir, d)) and d.startswith("Videos")
                ]
                subdirs.sort(key=str.lower)

                for subdir in subdirs:
                    full_path = os.path.join(root_dir, subdir)
                    for root, _, files in os.walk(full_path):
                        for file in sorted(files):
                            ext = os.path.splitext(file)[1].lower()
                            if ext in self.VIDEO_EXTENSIONS:
                                video_files.append(os.path.join(root, file))

            elif self.mode == "root":
                # 🔹 Lấy video trong root_dir và tất cả subdir bên trong
                for root, _, files in os.walk(root_dir):
                    for file in sorted(files):
                        ext = os.path.splitext(file)[1].lower()
                        if ext in self.VIDEO_EXTENSIONS:
                            video_files.append(os.path.join(root, file))

        return sorted(video_files)

    def split_into_n_batches(self, n_batches):
        """Chia list video thành n_batches (càng đều càng tốt)."""
        total = len(self.videos)
        if n_batches <= 0:
            raise ValueError("Số batch phải > 0")
        if total == 0:
            return []

        batch_size = math.ceil(total / n_batches)
        batches = []
        for i in range(0, total, batch_size):
            batches.append(self.videos[i:i + batch_size])
        return batches

# Thực thi

In [10]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("huggingface")

folder_path = '/kaggle/input/aic2025-batch-2'
prepare = VideoPathPrepare(folder_path, mode = 'subdir')

print(f"Tổng số video: {len(prepare.videos)}")

Tổng số video: 605


In [ ]:
from multi_gpu_extract import process_video
import os
if __name__ == "__main__":
    mp.set_start_method("spawn", force=True)
    video_list = prepare.videos[204:221]
    
    # Queue
    video_queue = mp.Manager().Queue()
    for v in video_list:
        video_queue.put(v)
        
    processes = []
    for gpu_id in [1, 0]:
        p = mp.Process(target=process_video, args=(gpu_id, video_queue, token))
        p.start()
        processes.append(p)
    
    for p in processes:
        p.join()
    
    print("🎉 All videos processed!")

🚀 Worker GPU 1 started
🎬 GPU 1 processing K07_V023
🚀 Worker GPU 0 started
🎬 GPU 0 processing K07_V022


Extracting /kaggle/input/aic2025-batch-2/Videos_K07/video/K07_V023.mp4:   0%|          | 9/26220 [00:00<15:32, 28.10it/s]
Extracting /kaggle/input/aic2025-batch-2/Videos_K07/video/K07_V022.mp4:   0%|          | 9/35918 [00:00<20:56, 28.59it/s]
🔄 Extracting embeddings (DataLoader):   0%|          | 0/1 [00:00<?, ?it/s]/kaggle/working/unilm/beit3/multi_gpu_extract.py:181: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/kaggle/working/unilm/beit3/multi_gpu_extract.py:181: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
🔄 Extracting embeddings (DataLoader): 100%|██████████| 1/1 [00:27<00:00, 27.74s/it]
No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.
Extracting /kaggle/input/aic2025-batch-2/Vi

Thời gian extract embedding: 27.55874514579773
✅ Saved /kaggle/working/embeddings/K07_V022.pkl
Videos_K07/K07_V022.pkl
🗑️ Deleted frames folder: /kaggle/working/frames_K07_V022
🎬 GPU 0 processing K07_V024


Extracting /kaggle/input/aic2025-batch-2/Videos_K07/video/K07_V024.mp4:   0%|          | 9/29265 [00:00<12:27, 39.15it/s]
🔄 Extracting embeddings (DataLoader):   0%|          | 0/1 [00:00<?, ?it/s]No files have been modified since last commit. Skipping to prevent empty commit.
Extracting /kaggle/input/aic2025-batch-2/Videos_K07/video/K07_V025.mp4:   0%|          | 0/31836 [00:00<?, ?it/s]

Thời gian extract embedding: 27.744462490081787
✅ Saved /kaggle/working/embeddings/K07_V023.pkl
Videos_K07/K07_V023.pkl
Đẩy file /kaggle/working/embeddings/K07_V023.pkl bị lỗi... Đang thử lại lần 1
🗑️ Deleted frames folder: /kaggle/working/frames_K07_V023
🎬 GPU 1 processing K07_V025


...

Extracting /kaggle/input/aic2025-batch-2/Videos_K08/video/K08_V003.mp4:   0%|          | 9/34191 [00:00<15:54, 35.81it/s]
🔄 Extracting embeddings (DataLoader):   0%|          | 0/1 [00:00<?, ?it/s]No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.
Extracting /kaggle/input/aic2025-batch-2/Videos_K08/video/K08_V004.mp4:   0%|          | 0/32168 [00:00<?, ?it/s]

Thời gian extract embedding: 24.84357762336731
✅ Saved /kaggle/working/embeddings/K08_V002.pkl
Videos_K08/K08_V002.pkl
🗑️ Deleted frames folder: /kaggle/working/frames_K08_V002
🎬 GPU 1 processing K08_V004


Extracting /kaggle/input/aic2025-batch-2/Videos_K08/video/K08_V004.mp4:   0%|          | 9/32168 [00:00<24:26, 21.93it/s]
🔄 Extracting embeddings (DataLoader): 100%|██████████| 1/1 [00:23<00:00, 23.06s/it]No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.
🔄 Extracting embeddings (DataLoader): 100%|██████████| 1/1 [00:24<00:00, 24.73s/it]   | 0/29336 [00:00<?, ?it/s]


Thời gian extract embedding: 24.898885250091553
✅ Saved /kaggle/working/embeddings/K08_V003.pkl
Videos_K08/K08_V003.pkl
🗑️ Deleted frames folder: /kaggle/working/frames_K08_V003
🎬 GPU 0 processing K08_V005


Extracting /kaggle/input/aic2025-batch-2/Videos_K08/video/K08_V005.mp4:   0%|          | 9/29336 [00:00<12:03, 40.53it/s]
🔄 Extracting embeddings (DataLoader):   0%|          | 0/1 [00:00<?, ?it/s]